# k-neares-neighbour (KNN)
The  KNN classifier is one of the most simples classifier to imagine. It is a nonparmametric method. The idea is that new instances get the same label as their neighbour from which the label is known. If compared to k neigbours, a majority vote can improve the prediction. 

In [1]:
using DataFrames

In [6]:
df = readtable("../Datasets/wdbc.data.txt", header=false)
df[:,end+1] = zeros(Int,size(df,1))
df[df[:,2] .== "M",end] = 1
delete!(df, 1)
delete!(df, 1)
data = convert(Array, df)
x = data[:,1:end-1]
y = data[:,end]
index = shuffle!(collect(1:size(y,1)))
i_train = index[1:round(Int,length(index)*0.6)]
i_test = index[round(Int,length(index)*0.6)+1:end]
y_train = y[i_train]
y_test = y[i_test]
x_train = Matrix{Float64}(x[i_train,:])
x_test = Matrix{Float64}(x[i_test,:])

228x30 Array{Float64,2}:
 11.71   17.19   74.68   420.3  0.09774  …  0.1521   0.1099   0.2572  0.07097
 10.29   27.61   65.67   321.4  0.0903      0.2      0.09127  0.2226  0.08283
 10.57   20.22   70.15   338.3  0.09073     0.603    0.1465   0.2597  0.12   
 14.04   15.98   89.78   611.2  0.08458     0.1117   0.07453  0.2725  0.07234
 13.7    17.64   87.76   571.1  0.0995      0.1742   0.09077  0.2518  0.0696 
 27.42   26.27  186.9   2501.0  0.1084   …  0.6833   0.2625   0.2641  0.07427
 13.77   13.27   88.06   582.7  0.09198     0.03732  0.05802  0.2823  0.06794
 15.08   25.74   98.0    716.6  0.1024      0.4029   0.1526   0.2654  0.09438
 16.24   18.77  108.8    805.1  0.1066      0.5026   0.1732   0.277   0.1063 
 13.0    20.78   83.51   519.4  0.1135      0.08112  0.06296  0.3196  0.06435
 17.02   23.98  112.8    899.3  0.1197   …  0.5588   0.1847   0.353   0.08482
 20.13   28.25  131.2   1261.0  0.0978      0.3215   0.1628   0.2572  0.06637
 12.07   13.44   77.83   445.2  0.11   

##  Distance calculation
To find ot what are the k closesed neighbours, we must compute the distance between the unknown and the known  instances.  Here we will just use the euklidean distance for simplicity but in principle every distance can be used.


In [7]:
function dist(X1::Matrix{Float64},X2::Matrix{Float64})
    X1_norm = diag(X1*X1.')
    X2_norm = diag(X2*X2.')
    X1_norm = repeat(X1_norm,outer =[1,size(X2)[1]])
    X2_norm = repeat(X2_norm,outer =[1,size(X1)[1]])
    return sqrt(X1_norm .+ X2_norm.' .- 2.* X1 * X2.')
end

dist (generic function with 1 method)

In [8]:
@time dist(x_train,x_train)
@time dist(x_train,x_train)

  3.585894 seconds (3.38 M allocations: 131.319 MB, 2.34% gc time)
  0.397413 seconds (2.09 M allocations: 68.369 MB, 6.77% gc time)


341x341 Array{Float64,2}:
    0.0    1631.23   2108.8     2114.77    …   296.634  2265.7      579.016
 1631.23      0.0     479.717    489.795      1420.0     638.155   1088.85 
 2108.8     479.717     0.0       59.2346     1893.53    159.295   1563.45 
 2114.77    489.795    59.2346     0.0        1893.98    159.401   1564.52 
 2080.21    453.193    49.54      41.1644     1861.36    188.95    1531.53 
  419.575  1737.95   2202.85    2198.11    …   358.507  2355.02     665.566
 1495.61    200.071   634.195    630.297      1265.76    787.131    936.185
 1968.76    343.924   147.281    146.758      1749.63    299.608   1419.97 
 1765.88    198.706   373.563    362.242      1536.86    520.097   1208.76 
 2288.02    659.767   180.199    183.924      2071.72     29.2635  1742.12 
  716.153  2180.16   2644.2     2638.72    …   783.522  2795.8     1103.94 
 1837.29    210.58    273.914    280.532      1620.27    431.142   1289.73 
  547.133  1285.96   1746.21    1740.3         252.351  1897.4

In [9]:
function dist1(X1::Matrix{Float64},X2::Matrix{Float64})
    dist = Matrix{Float64}(size(X1)[1],size(X2)[1])
    for j = 1:size(X2)[1]
        for i = 1:size(X1)[1]
            tmp = vec(X1[i,:] .- X2[j,:])
            dist[i,j] = dot(tmp,tmp)
        end
    end
    return sqrt(dist)
end

dist1 (generic function with 1 method)

In [10]:
@time dist1(x_train,x_train)
@time dist1(x_train,x_train)

  0.579663 seconds (1.69 M allocations: 169.731 MB, 8.45% gc time)
  0.406715 seconds (1.63 M allocations: 166.785 MB, 12.32% gc time)


341x341 Array{Float64,2}:
    0.0    1631.23   2108.8     2114.77    …   296.634  2265.7      579.016
 1631.23      0.0     479.717    489.795      1420.0     638.155   1088.85 
 2108.8     479.717     0.0       59.2346     1893.53    159.295   1563.45 
 2114.77    489.795    59.2346     0.0        1893.98    159.401   1564.52 
 2080.21    453.193    49.54      41.1644     1861.36    188.95    1531.53 
  419.575  1737.95   2202.85    2198.11    …   358.507  2355.02     665.566
 1495.61    200.071   634.195    630.297      1265.76    787.131    936.185
 1968.76    343.924   147.281    146.758      1749.63    299.608   1419.97 
 1765.88    198.706   373.563    362.242      1536.86    520.097   1208.76 
 2288.02    659.767   180.199    183.924      2071.72     29.2635  1742.12 
  716.153  2180.16   2644.2     2638.72    …   783.522  2795.8     1103.94 
 1837.29    210.58    273.914    280.532      1620.27    431.142   1289.73 
  547.133  1285.96   1746.21    1740.3         252.351  1897.4

In [52]:
function dist2(X1::Matrix{Float64},X2::Matrix{Float64})
    nrows = size(X1,1)
    ncols = size(X2,1)
    nfeat = size(X2,2)
    dist = Matrix{Float64}(nrows,ncols)
    for j = 1:ncols
        for i = 1:nrows
            dist[i,j] = 0
            for k = 1:nfeat
                dist[i,j] += (X1[i,k] - X2[j,k])^2
            end
            dist[i,j] = sqrt(dist[i,j])
        end
    end
    return dist
end

dist2 (generic function with 1 method)

In [53]:
@time dist2(x_train,x_train)
@time dist2(x_train,x_train)

  0.083811 seconds (7.92 k allocations: 1.254 MB)
  0.019542 seconds (6 allocations: 908.672 KB)


341x341 Array{Float64,2}:
    0.0    1631.23   2108.8     2114.77    …   296.634  2265.7      579.016
 1631.23      0.0     479.717    489.795      1420.0     638.155   1088.85 
 2108.8     479.717     0.0       59.2346     1893.53    159.295   1563.45 
 2114.77    489.795    59.2346     0.0        1893.98    159.401   1564.52 
 2080.21    453.193    49.54      41.1644     1861.36    188.95    1531.53 
  419.575  1737.95   2202.85    2198.11    …   358.507  2355.02     665.566
 1495.61    200.071   634.195    630.297      1265.76    787.131    936.185
 1968.76    343.924   147.281    146.758      1749.63    299.608   1419.97 
 1765.88    198.706   373.563    362.242      1536.86    520.097   1208.76 
 2288.02    659.767   180.199    183.924      2071.72     29.2635  1742.12 
  716.153  2180.16   2644.2     2638.72    …   783.522  2795.8     1103.94 
 1837.29    210.58    273.914    280.532      1620.27    431.142   1289.73 
  547.133  1285.96   1746.21    1740.3         252.351  1897.4

LoadError: LoadError: UndefVarError: view not defined
while loading In[49], in expression starting on line 1

In [42]:
using Distances

@time pairwise(Euclidean(), x_train.',x_train.')
@time pairwise(Euclidean(), x_train.',x_train.')

  0.006374 seconds (2.06 k allocations: 1.122 MB)
  0.002511 seconds (2.06 k allocations: 1.122 MB)


341x341 Array{Float64,2}:
    4.31584e-5  1631.23   2108.8     …   296.634  2265.7      579.016
 1631.23           0.0     479.717      1420.0     638.155   1088.85 
 2108.8          479.717     0.0        1893.53    159.295   1563.45 
 2114.77         489.795    59.2346     1893.98    159.401   1564.52 
 2080.21         453.193    49.54       1861.36    188.95    1531.53 
  419.575       1737.95   2202.85    …   358.507  2355.02     665.566
 1495.61         200.071   634.195      1265.76    787.131    936.185
 1968.76         343.924   147.281      1749.63    299.608   1419.97 
 1765.88         198.706   373.563      1536.86    520.097   1208.76 
 2288.02         659.767   180.199      2071.72     29.2635  1742.12 
  716.153       2180.16   2644.2     …   783.522  2795.8     1103.94 
 1837.29         210.58    273.914      1620.27    431.142   1289.73 
  547.133       1285.96   1746.21        252.351  1897.47     273.171
    ⋮                                ⋱                          

In [54]:
function KNN(x_train, y_train, x_test, k=3)
    # first calculate distance bewteen train and test data
    d = dist(x_train, x_test)
    # find labels of closest neighbours
    labels = Matrix(size(x_test,1),k)
    for i = 1:k
        mxval, mxindx = findmin(d, 1)
        indx = ind2sub(size(d), vec(mxindx))[1]
        labels[:,i] = (y_train[indx])
        # remove closest values from distance matrix
        d[mxindx] = Inf
    end
    return median(labels,2)
end

KNN (generic function with 2 methods)

In [55]:
y_pred = KNN(x_train, y_train, x_test,5)
mean(y_pred .== y_test)


0.9254385964912281

In [57]:
y_pred = KNN(x_train, y_train, x_train,5)
mean(y_train .== y_pred)

0.9384164222873901